In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import sys
from sqlalchemy import create_engine
import urllib
import pyodbc
import requests
from time import sleep

In [ ]:
# https://www.oregon.gov/olcc/marijuana/Pages/Recreational-Marijuana-Licensing.aspx

In [ ]:
# Just so I see the whole DataFrame
pd.options.display.max_columns = 100
pd.options.display.min_rows = 100
pd.options.display.max_rows = 100

In [ ]:
# Creating a dt file of today's date
today = dt.date(2021, 9, 20)

In [ ]:
# This creates the 2 digit versions of the day, month, and year that will be used in file names
today_day = today.strftime("%d")
today_month = today.strftime("%m")
today_year = today.strftime("%Y")

### Creating SQL Connection

In [ ]:
# # This creates the connection to my local Oregon Cannabis files in SQL
# driver = "ODBC Driver 17 for SQL Server"
# engine = create_engine(f'mssql://LAPTOP-E6QKON1L/or_cannabis?driver={driver}')
# engine_con = engine.connect()

In [ ]:
# This creates the connection to the Bespoke Engine's Oregon Cannabis files in SQL
params = urllib.parse.quote_plus("DRIVER={ODBC Driver 17 for SQL Server};"
                                 "SERVER=bespoke-database-1.cmevrozrcs7c.us-west-2.rds.amazonaws.com;"
                                 "DATABASE=or_cannabis;"
                                 "UID=admin;"
                                 "PWD=N19lrqxnurTUJLJT6GFe")
engine = create_engine("mssql+pyodbc:///?odbc_connect={}".format(params))
engine_con = engine.connect()

## Read in Data

In [ ]:
# This reads in the data provided by https://www.oregon.gov/olcc/marijuana/Pages/Recreational-Marijuana-Licensing.aspx
all_licenses = pd.read_excel(f"license_repository/{today.year}_{today_month}_{today_day} Licenses.xlsx", header=2)

In [ ]:
# Couldn't use license number because of something that appeared at the bottom after a few nulls
all_licenses = all_licenses[all_licenses['LICENSEE NAME'].notnull()]

In [ ]:
# This changes the yes/nul columns to binary
all_licenses[['Retail Delivery', 'Medical Grade', 'Hemp']] = all_licenses[['Retail Delivery', 'Medical Grade', 'Hemp']].replace('Yes', 1).replace(np.nan, 0).astype(int)

In [ ]:
all_licenses.head()

### Data Cleaning

In [ ]:
# This cleans the spaces from the beggining and end of names and such

all_licenses['LICENSEE NAME'] = all_licenses['LICENSEE NAME'].str.strip()
all_licenses['BUSINESS NAME'] = all_licenses['BUSINESS NAME'].str.strip()
all_licenses['LICENSE NUMBER'] = all_licenses['LICENSE NUMBER'].str.strip().str.replace(' ', '-')

### Rename Data

In [ ]:
renamed_data = pd.DataFrame(data=all_licenses['LICENSE NUMBER'].rename('license_number'))

renamed_data['license_description'] = all_licenses['LICENSE TYPE']

renamed_data['name_legal'] = all_licenses['LICENSEE NAME']
renamed_data['name_dba'] = all_licenses['BUSINESS NAME']

renamed_data['contact_county'] = all_licenses['COUNTY']
renamed_data['contact_state'] = 'OR'

renamed_data['license_type_retail_delivery'] = all_licenses['Retail Delivery']
renamed_data['license_type_medical_grade'] = all_licenses['Medical Grade']
renamed_data['license_type_hemp'] = all_licenses['Hemp']

renamed_data['is_current'] = 1

renamed_data['date_uploaded'] = today

In [ ]:
renamed_data

In [ ]:
pd.read_sql('or_main', con=engine_con)

### Merging Info From SQL

In [ ]:
# This reads in the databases so that can be merged
sql_old_main = pd.read_sql('or_main', con=engine_con)
sql_old_roll = pd.read_sql('or_roll', con=engine_con)
sql_old_contact = pd.read_sql('or_contact', con=engine_con)

In [ ]:
# This merges in the roll up information for companies that have already been given company roll_ups and roll_up_ids
df_with_roll = renamed_data.merge(sql_old_roll, how='left')

In [ ]:
# This merges in the contact information for companies from SQL
df_with_contact = df_with_roll.merge(sql_old_contact, how='left')

In [ ]:
# This will be fiddled with in the long run, but will create the csv for company_roll_ups to be added to the first time
df_for_csv = df_with_contact[[
              'license_number', 'license_description', 'name_legal', 'name_dba', 'roll_up_id', 'company_roll_up',
              'contact_email', 'contact_website', 'contact_phone', 'contact_county', 'contact_state',
              'license_type_retail_delivery', 'license_type_medical_grade', 'license_type_hemp', 'is_current', 'date_uploaded'
                            ]]

In [ ]:
# This drops the Nones from the contact info columns
df_for_csv[['contact_email', 'contact_website', 'contact_phone']] = df_for_csv[['contact_email', 'contact_website', 'contact_phone']].fillna('')

In [ ]:
df_for_csv['is_current'].value_counts()

### Create CSV to Add Company Roll Up

In [ ]:
df_for_csv.sort_values(by='name_dba', inplace=True)

In [ ]:
df_for_csv.to_csv(f"edited_files/df_{today_year}_{today_month}_{today_day}_to_edit.csv", index=False)

#### Add company_roll_ups Where Needed then Read Back In

In [ ]:
# This reads the file back in after checking that no company_roll_ups were missed
# Switch to f"edited_files/df_to_edit_{today_month}_{today_day}_{today_year}.csv" after SQL is done

if pd.read_csv(f"edited_files/df_{today_year}_{today_month}_{today_day}_to_edit.csv")['company_roll_up'].isnull().sum() > 0:
    print(error)
else:
    df_edited = pd.read_csv(f"edited_files/df_{today_year}_{today_month}_{today_day}_to_edit.csv")

In [ ]:
df_edited.head(3)

### Contact Info

In [ ]:
# Since I'm trying to add the contact info through Excel, I'll clean it here before continuing

In [ ]:
df_contact_edit = df_edited

In [ ]:
# This cleans the contact_phone section

df_contact_edit['contact_phone'] = df_contact_edit['contact_phone'].astype(str).replace('tel','')
df_contact_edit['contact_phone'] = df_contact_edit['contact_phone'].astype(str).replace('-','')
df_contact_edit['contact_phone'] = df_contact_edit['contact_phone'].astype(str).replace('+','')
df_contact_edit['contact_phone'] = df_contact_edit['contact_phone'].astype(str).replace('.','')
df_contact_edit['contact_phone'] = df_contact_edit['contact_phone'].astype(str).replace('?','')
df_contact_edit['contact_phone'] = df_contact_edit['contact_phone'].astype(str).replace('!','')
df_contact_edit['contact_phone'] = df_contact_edit['contact_phone'].astype(str).replace(':','')
df_contact_edit['contact_phone'] = df_contact_edit['contact_phone'].astype(str).replace('(','')
df_contact_edit['contact_phone'] = df_contact_edit['contact_phone'].astype(str).replace(')','')
df_contact_edit['contact_phone'] = df_contact_edit['contact_phone'].astype(str).replace('\n','')
df_contact_edit['contact_phone'] = df_contact_edit['contact_phone'].astype(str).replace(' ','')

In [ ]:
# Writing code for this would take ot long, so just gonna replace it manually
df_contact_edit['contact_phone'] = df_contact_edit['contact_phone'].str.replace('1833RIPWEED', '18337479333')
df_contact_edit['contact_phone'] = df_contact_edit['contact_phone'].str.replace('35385219753', '')

In [ ]:
# This will enable a little bit more cleaning of each

all_phones = list(df_contact_edit['contact_phone'])
all_emails = list(df_contact_edit['contact_email'])
all_websites = list(df_contact_edit['contact_website'])

In [ ]:
# This drops all of the phone numbers that are too short to be phone numbers

for x in range(len(all_phones)):
    if len(str(all_phones[x])) < 10:
        all_phones[x] = ''

In [ ]:
# This drops all of the phone numbers that are too long to be phone numbers

for x in range(len(all_phones)):
    if len(str(all_phones[x])) > 11:
        all_phones[x] = ''

In [ ]:
# This drops all of the instragram accounts and other non-email methods of contact

for x in range(len(all_emails)):
    if '@' not in str(all_emails[x]):
        all_emails[x] = ''

In [ ]:
# This drops all of the instragram accounts

for x in range(len(all_websites)):
    if 'insta' in str(all_websites[x]):
        all_websites[x] = ''

In [ ]:
# This drops all of the facebook accounts

for x in range(len(all_websites)):
    if 'facebook' in str(all_websites[x]):
        all_websites[x] = ''

In [ ]:
# And then we just add the edited lists back in to the dataframe

df_contact_edit['contact_phone'] = all_phones
df_contact_edit['contact_email'] = all_emails
df_contact_edit['contact_website'] = all_websites

In [ ]:
# Just filling the NaNs

df_contact_edit['contact_phone'] = df_contact_edit['contact_phone'].fillna('')
df_contact_edit['contact_email'] = df_contact_edit['contact_email'].fillna('')
df_contact_edit['contact_website'] = df_contact_edit['contact_website'].fillna('')

In [ ]:
df_contact_edit

### Checking if Licenses Still Current

In [ ]:
# Idk why this turned itself back into a Boolean, but now it's numbers again
sql_old_main['is_current'] = sql_old_main['is_current'].astype(int)

In [ ]:
# This creates the lists of old and new license number
old_lic_list = list(sql_old_main.license_number)
new_lic_list = list(df_contact_edit.license_number)

In [ ]:
# This creates the list of licenses that are no longer current
not_current_list = []
for lic in old_lic_list:
    if lic not in new_lic_list:
        not_current_list.append(lic)

In [ ]:
not_current_list

In [ ]:
len(sql_old_main)

In [ ]:
len(df_edited)

In [ ]:
sql_old_main

In [ ]:
# This creates a df of the licenses that aren't current, changes the is_current to 0, and appends the df to the end of 
not_current_df = pd.DataFrame(not_current_list, columns={'license_number'})
not_current_df = not_current_df.merge(sql_old_main, how='left').merge(sql_old_roll, how='left').merge(sql_old_contact, how='left')
not_current_df['is_current'] = 0
df_edited_current = df_contact_edit.append(not_current_df).sort_values(by='name_legal').reset_index().drop(columns=['index'])

In [ ]:
# Just a quick check
df_edited_current[df_edited_current['is_current'] == 0]

In [ ]:
df_edited_current.is_current.value_counts()

### Creating a Dataframe for Making Changes to SQL

In [ ]:
df_edited_current.head(3)

In [ ]:
sql_old_main.columns

In [ ]:
to_alter = []

for lic in df_edited_current['license_number']:
    if list(df_edited_current[df_edited_current['license_number'] == lic]['license_description'])[0] != list(sql_old_main[sql_old_main['license_number'] == lic]['license_description'])[0]:
        to_alter.append(lic)
    elif list(df_edited_current[df_edited_current['license_number'] == lic]['license_type_retail_delivery'])[0] != list(sql_old_main[sql_old_main['license_number'] == lic]['license_type_retail_delivery'])[0]:
        to_alter.append(lic)
    elif list(df_edited_current[df_edited_current['license_number'] == lic]['license_type_medical_grade'])[0] != list(sql_old_main[sql_old_main['license_number'] == lic]['license_type_medical_grade'])[0]:
        to_alter.append(lic)
    elif list(df_edited_current[df_edited_current['license_number'] == lic]['license_type_hemp'])[0] != list(sql_old_main[sql_old_main['license_number'] == lic]['license_type_hemp'])[0]:
        to_alter.append(lic)
    elif list(df_edited_current[df_edited_current['license_number'] == lic]['is_current'])[0] != list(sql_old_main[sql_old_main['license_number'] == lic]['is_current'])[0]:
        to_alter.append(lic)
    
df_for_alter = df_edited_current[df_edited_current['license_number'].isin(to_alter)]

In [ ]:
df_for_alter.shape

In [ ]:
# # This creates a list of all of the license_numbers of rows that have a change in them

# df_for_alter = df_edited_current
# to_alter = []

# # There's probably a cleaner way to do this, but this gets the list of all licenses where there is some change in information
# for lic in sql_old_main['license_number']:
#     if 'False' in str(df_for_alter[df_for_alter['license_number'] == lic][['license_number', 'name_legal', 'name_dba', 'license_description', 'license_type_retail_delivery', 'license_type_medical_grade', 'license_type_hemp', 'is_current']].values == sql_old_main[sql_old_main['license_number'] == lic][['license_number', 'name_legal', 'name_dba', 'license_description', 'license_type_retail_delivery', 'license_type_medical_grade', 'license_type_hemp', 'is_current']].values):
#         to_alter.append(lic)
        
# # This may be only needed the first time
# for x in range(len(df_for_alter)):
#     if df_for_alter.loc[x]['is_current'] == 0:
#         to_alter.append(df_for_alter.loc[x]['license_number'])

In [ ]:
df_for_alter

In [ ]:
# And then we turn that list into a DataFrame
main_alter = pd.DataFrame(list(to_alter), columns=['license_number']).merge(df_for_alter, how='left')

In [ ]:
main_alter

In [ ]:
# And let's toss this into a csv/sql

# main_alter.to_excel(f'edited_files/main_alter_{today.year}_{today_month}_{today_day}.xlsx', index=False, sheet_name='or_main_alter')
main_alter.to_sql('or_main_alter', con=engine_con, if_exists='replace', index=False)

#### Back to Cleaning the Data

In [ ]:
# This creates two seperate dataframes for rows needing roll_up_ids and those that don't

df_edited_null = df_edited[df_edited['roll_up_id'].isnull()]
df_edited_fill = df_edited[df_edited['roll_up_id'].notnull()]

In [ ]:
# This creates all the roll_up_id numbers

new_comp = list(df_edited_null['company_roll_up'].unique())

max_roll = df_edited_fill['roll_up_id'].map(lambda x: x.lstrip('OR-')).astype(int).max()
# max_roll = 0 # Only for used the first time
    
new_roll = []
for n in range(len(new_comp)):
    max_roll += 1
    new_roll.append(max_roll)
    
new_or_roll = []
for num in new_roll:
    num = str(num)
    
    # This should hopefully add the 0s properly
    while len(num) < 5:
        num = '0' + num
        
    new_or_roll.append(f'OR-{num}')

In [ ]:
df_edited_null

In [ ]:
# This uses the two lists created above to add roll_up_ids to the matching company_roll_ups to the df_edited

for n in list(df_edited_null.index):
    df_edited_null['roll_up_id'][n] = new_or_roll[new_comp.index(df_edited_null['company_roll_up'][n])]

In [ ]:
df_edited_null.head(3)

In [ ]:
# Probably more stuff will go here after the first upload

In [ ]:
# This reattaches the two dataframes now that the roll_up_ids have been filled

df_populated = df_edited_fill.append(df_edited_null)

In [ ]:
df_populated

In [ ]:
df_populated.columns

In [ ]:
# This is to serperate out the licenses that aren't already in the SQL database

lic_df = list(df_populated['license_number'].unique())
lic_sql = list(sql_old_roll['license_number'].unique())
lic_new = []

for lic in lic_df:
    if lic not in lic_sql:
        lic_new.append(lic)

lic_new

In [ ]:
# This creates the final dataframe for sending data to SQL

df_to_add = pd.DataFrame(lic_new, columns=['license_number'])
df_to_add = df_to_add.merge(df_populated, how='left')

In [ ]:
# This adds the current date to the new data being added to SQL
df_to_add['date_uploaded'] = today

In [ ]:
df_to_add

### Seperate Data for SQL

In [ ]:
# This just seperates the dataframe into the main and roll information that will be put into SQL

df_or_main = df_to_add[['license_number', 'name_legal', 'name_dba', 'license_description', 'license_type_retail_delivery', 'license_type_medical_grade', 'license_type_hemp', 'is_current', 'date_uploaded']]

df_or_roll = df_to_add[['license_number', 'roll_up_id', 'company_roll_up']]

df_or_contact = df_to_add[['license_number', 'contact_email', 'contact_website', 'contact_phone', 'contact_county', 'contact_state']]

In [ ]:
# # This turned all of the roll_up_ids into 5 digit numbers so they'll actually list in order properly
# # Probably won't need to rerun, but might as well leave here
# new_roll_up_id_list = []
# for roll in list(df_or_roll.roll_up_id):
#     roll_num = str(roll)[3:]
#     while len(roll_num) < 5:
#         roll_num = '0' + roll_num
#     new_roll_up_id_list.append(str('OR-' + roll_num))

### To SQL

In [ ]:
df_or_roll.to_sql('or_roll', con=engine_con, if_exists='append', index=False)

In [ ]:
df_or_main.to_sql('or_main', con=engine_con, if_exists='append', index=False)

In [ ]:
df_or_contact.to_sql('or_contact', con=engine_con, if_exists='append', index=False)

In [ ]:
df_or_main

In [ ]:
df_populated

### Add to all_main

In [ ]:
# This creates the connection to the all_main file in SQL

all_cannabis_params = urllib.parse.quote_plus("DRIVER={ODBC Driver 17 for SQL Server};"
                                 "SERVER=bespoke-database-1.cmevrozrcs7c.us-west-2.rds.amazonaws.com;"
                                 "DATABASE=all_cannabis;"
                                 "UID=admin;"
                                 "PWD=N19lrqxnurTUJLJT6GFe")
all_cannabis_con = create_engine("mssql+pyodbc:///?odbc_connect={}".format(all_cannabis_params))

In [ ]:
# This reads in the all_main data and narrows it down to the relevent data
all_main_sql = pd.read_sql('all_main', con=all_cannabis_con)
old_or_main_sql = all_main_sql[all_main_sql['us_state'] == 'OR']
old_or_main_sql = old_or_main_sql.reset_index().drop(columns='index')

In [ ]:
# This re-reads in and merges the ca_annabis data in SQL
new_or_main = pd.read_sql('or_main', con=engine_con)
new_or_roll = pd.read_sql('or_roll', con=engine_con)
or_with_roll = new_or_main.merge(new_or_roll, on='license_number')

In [ ]:
old_or_main_sql

In [ ]:
or_with_roll

In [ ]:
or_with_roll['license_category'] = ''
or_with_roll['license_status'] = ''
or_with_roll['date_expiration'] = None

In [ ]:
for n in range(len(or_with_roll)):
    if or_with_roll['license_description'][n] in ['Recreational Producer']:
        or_with_roll['license_category'][n] = 'Cultivator'
    if or_with_roll['license_description'][n] in ['Recreational Retailer']:
        or_with_roll['license_category'][n] = 'Retailer'
    if or_with_roll['license_description'][n] in ['Recreational Processor']:
        or_with_roll['license_category'][n] = 'Manufacturer'
    if or_with_roll['license_description'][n] in ['Recreational Wholesaler']:
        or_with_roll['license_category'][n] = 'Distributor'
    if or_with_roll['license_description'][n] in ['Laboratory']:
        or_with_roll['license_category'][n] = 'Laboratory'
    if or_with_roll['license_description'][n] in ['Research Certificate']:
        or_with_roll['license_category'][n] = 'Other'

In [ ]:
# This reformats the data for the all_main
new_main_for_sql = or_with_roll[['license_number', 'roll_up_id', 'name_legal', 'license_status', 'is_current', 'license_description', 'date_expiration', 'license_category']]
new_main_for_sql['state'] = 'OR'

In [ ]:
new_main_for_sql

In [ ]:
# Renaming the data, since we changed the column names
new_main_for_sql = new_main_for_sql.rename(columns={
    'license_number':'license_number',
    'roll_up_id':'rollup_id',
    'name_legal':'legal_name',
    'license_status':'license_status',
    'is_current':'is_current',
    'license_description':'license_description',
    'state':'us_state',
    'license_sub_category':'license_category',
    'date_expiration':'expiration_date'
})

In [ ]:
new_or_for_sql = new_main_for_sql[~new_main_for_sql['license_number'].isin(list(old_or_main_sql['license_number'].unique()))]

In [ ]:
new_or_for_sql

In [ ]:
# And this uploads the new licenses to SQL
new_or_for_sql.to_sql('all_main', con=all_cannabis_con, index=False, if_exists='append')

### Alter the OR data in all_main

In [ ]:
# This re-reads in the Oregon database to check for any need to alter
or_main_sql_for_alter = pd.read_sql('or_main', con=engine_con)
or_roll_sql_for_alter = pd.read_sql('or_roll', con=engine_con)
or_roll_sql_for_alter = or_roll_sql_for_alter
or_sql_data = or_main_sql_for_alter.merge(or_roll_sql_for_alter, on='license_number').rename(columns={'roll_up_id': 'rollup_id', 'name_legal': 'legal_name'})

In [ ]:
# This reads in the all_main database and narrows in down to California licenses to do the same
all_main_sql_for_alter = pd.read_sql('all_main', con=all_cannabis_con)
or_main_sql_for_alter = all_main_sql_for_alter[all_main_sql_for_alter['us_state'] == 'OR']
or_main_sql_for_alter = or_main_sql_for_alter.reset_index().drop(columns='index')

In [ ]:
or_sql_data['license_category'] = ''

for n in range(len(or_with_roll)):
    if or_sql_data['license_description'][n] in ['Recreational Producer']:
        or_sql_data['license_category'][n] = 'Cultivator'
    if or_sql_data['license_description'][n] in ['Recreational Retailer']:
        or_sql_data['license_category'][n] = 'Retailer'
    if or_sql_data['license_description'][n] in ['Recreational Processor']:
        or_sql_data['license_category'][n] = 'Manufacturer'
    if or_sql_data['license_description'][n] in ['Recreational Wholesaler']:
        or_sql_data['license_category'][n] = 'Distributor'
    if or_sql_data['license_description'][n] in ['Laboratory']:
        or_sql_data['license_category'][n] = 'Laboratory'
    if or_sql_data['license_description'][n] in ['Research Certificate']:
        or_sql_data['license_category'][n] = 'Other'

In [ ]:
or_sql_data

In [ ]:
or_all_main_alter_list = []

for lic in or_sql_for_alter['license_number']:
    if list(or_sql_for_alter[or_sql_for_alter['license_number'] == lic]['rollup_id'])[0] != list(or_sql_data[or_sql_data['license_number'] == lic]['rollup_id'])[0]:
        print(lic, 'rollup')
        or_all_main_alter_list.append(lic)
    elif list(or_sql_for_alter[or_sql_for_alter['license_number'] == lic]['license_category'])[0] != list(or_sql_data[or_sql_data['license_number'] == lic]['license_category'])[0]:
        print(lic, 'category')
        or_all_main_alter_list.append(lic)
    elif list(or_sql_for_alter[or_sql_for_alter['license_number'] == lic]['is_current'])[0] != list(or_sql_data[or_sql_data['license_number'] == lic]['is_current'])[0]:
        print(lic, 'current')
        or_all_main_alter_list.append(lic)
    elif list(or_sql_for_alter[or_sql_for_alter['license_number'] == lic]['legal_name'])[0] != list(or_sql_data[or_sql_data['license_number'] == lic]['legal_name'])[0]:
        print(lic, 'name')
        or_all_main_alter_list.append(lic)
        
# or_all_main_alter_list

In [ ]:
or_all_main_alter_df = or_sql_data[or_sql_data['license_number'].isin(or_all_main_alter_list)]
or_all_main_alter_df.shape

In [ ]:
or_all_main_alter_df['us_state'] = 'OR'

In [ ]:
or_all_main_alter_df = or_all_main_alter_df[['license_number', 'rollup_id', 'legal_name', 'is_current', 'license_description', 'us_state', 'license_category']]

In [ ]:
or_all_main_alter_df

In [ ]:
# This sends the df of data to alter into a state specific sql table
or_all_main_alter_df.to_sql('or_to_alter', con=all_cannabis_con, index=False, if_exists='replace')

In [ ]:
# And this code runs the query to update the sql_main table with the ca_to_alter table

all_conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                      'SERVER=bespoke-database-1.cmevrozrcs7c.us-west-2.rds.amazonaws.com;'
                      'DATABASE=all_cannabis;'
                      'UID=admin;'
                      'PWD=N19lrqxnurTUJLJT6GFe;')

all_cursor = all_conn.cursor()

all_cursor.execute('SELECT * FROM all_cannabis.dbo.all_main')

all_cursor.execute('''
Update t1
SET t1.[license_description] = t2.[license_description],
    t1.[license_category] = t2.[license_category],
    t1.[is_current] = t2.[is_current]
FROM [all_cannabis].[dbo].[all_main] as t1
INNER JOIN [all_cannabis].[dbo].[or_to_alter] as t2
ON t1.[license_number] = t2.[license_number];
''')

all_conn.commit()

### Bespoke / Sweat Equity API

In [ ]:
new_main_for_api.shape

In [ ]:
new_main_for_api = new_or_for_sql.append(or_all_main_alter_df)

In [ ]:
new_main_for_api['license_status'] = ''
new_main_for_api['expiration_date'] = None

In [ ]:
new_main_for_api

In [ ]:
new_main_for_api[new_main_for_api['license_number'] == '050 1011139BD62']

In [ ]:
new_main_for_api = new_main_for_api.reset_index().drop(columns='index')

In [ ]:
new_main = pd.read_sql()

In [ ]:
for n in range(len(new_main_for_api)):
    
    if new_main_for_api['is_current'].iloc[n] == 1:
        is_current_1 = True
    else:
        is_current_1 = False
        
        
    requests.post(
#             'https://bespoke-api-server-staging.herokuapp.com/api/v1/licenses/bulk_update_licenses',
            'https://bespoke-api-server-production.herokuapp.com/api/v1/licenses/bulk_update_licenses',
            json= {
          'company_licenses': [
              
              {
            'license_number': new_main_for_api['license_number'][n],
            'rollup_id': new_main_for_api['rollup_id'][n],
            'legal_name': new_main_for_api['legal_name'][n],
            'license_status': new_main_for_api['license_status'][n],
            'is_current': is_current_1,
            'license_description': new_main_for_api['license_description'][n],
            'us_state': new_main_for_api['us_state'][n],
            'expiration_date': new_main_for_api['expiration_date'][n],
            'license_category': new_main_for_api['license_category'][n]
            }
              
              
    
              
          ],
        },
            headers={'x-api-key': 'ASYNC-SERVER-API-KEY-6Lfy84qn9ew93ZwMXSYhdwqj'}
#             {'x-api-key': '80030179832729980933'}
        )
        
    sleep(5)

In [ ]:
print(stop)

In [ ]:
num_of_lic_sent = 0

while len(new_main_for_api) - num_of_lic_sent >= 10:

    for n in range(num_of_lic_sent, num_of_lic_sent+10):
    
        if new_main_for_api['is_current'].iloc[n] == 1:
            is_current_1 = True
        else:
            is_current_1 = False
            
        if new_main_for_api['is_current'].iloc[n+1] == 1:
            is_current_2 = True
        else:
            is_current_2 = False
            
        if new_main_for_api['is_current'].iloc[n+2] == 1:
            is_current_3 = True
        else:
            is_current_3 = False
            
        if new_main_for_api['is_current'].iloc[n+3] == 1:
            is_current_4 = True
        else:
            is_current_4 = False
            
        if new_main_for_api['is_current'].iloc[n+4] == 1:
            is_current_5 = True
        else:
            is_current_5 = False
            
        if new_main_for_api['is_current'].iloc[n+5] == 1:
            is_current_6 = True
        else:
            is_current_6 = False
            
        if new_main_for_api['is_current'].iloc[n+6] == 1:
            is_current_7 = True
        else:
            is_current_7 = False
            
        if new_main_for_api['is_current'].iloc[n+7] == 1:
            is_current_8 = True
        else:
            is_current_8 = False
            
        if new_main_for_api['is_current'].iloc[n+8] == 1:
            is_current_9 = True
        else:
            is_current_9 = False
            
        if new_main_for_api['is_current'].iloc[n+9] == 1:
            is_current_10 = True
        else:
            is_current_10 = False
            

        requests.post(
#             'https://bespoke-api-server-staging.herokuapp.com/api/v1/licenses/bulk_update_licenses',
            'https://bespoke-api-server-production.herokuapp.com/api/v1/licenses/bulk_update_licenses',
            json= {
          'company_licenses': [
              
              {
            'license_number': new_main_for_api['license_number'][n],
            'rollup_id': new_main_for_api['rollup_id'][n],
            'legal_name': new_main_for_api['legal_name'][n],
            'license_status': new_main_for_api['license_status'][n],
            'is_current': is_current_1,
            'license_description': new_main_for_api['license_description'][n],
            'us_state': new_main_for_api['us_state'][n],
            'expiration_date': new_main_for_api['expiration_date'][n],
            'license_category': new_main_for_api['license_category'][n]
            },
            
              
              {
            'license_number': new_main_for_api['license_number'][n+1],
            'rollup_id': new_main_for_api['rollup_id'][n+1],
            'legal_name': new_main_for_api['legal_name'][n+1],
            'license_status': new_main_for_api['license_status'][n+1],
            'is_current': is_current_2,
            'license_description': new_main_for_api['license_description'][n+1],
            'us_state': new_main_for_api['us_state'][n+1],
            'expiration_date': new_main_for_api['expiration_date'][n+1],
            'license_category': new_main_for_api['license_category'][n+1]
            },
              
              
              {
            'license_number': new_main_for_api['license_number'][n+2],
            'rollup_id': new_main_for_api['rollup_id'][n+2],
            'legal_name': new_main_for_api['legal_name'][n+2],
            'license_status': new_main_for_api['license_status'][n+2],
            'is_current': is_current_3,
            'license_description': new_main_for_api['license_description'][n+2],
            'us_state': new_main_for_api['us_state'][n+2],
            'expiration_date': new_main_for_api['expiration_date'][n+2],
            'license_category': new_main_for_api['license_category'][n+2]
            },

              
              {
            'license_number': new_main_for_api['license_number'][n+3],
            'rollup_id': new_main_for_api['rollup_id'][n+3],
            'legal_name': new_main_for_api['legal_name'][n+3],
            'license_status': new_main_for_api['license_status'][n+3],
            'is_current': is_current_4,
            'license_description': new_main_for_api['license_description'][n+3],
            'us_state': new_main_for_api['us_state'][n+3],
            'expiration_date': new_main_for_api['expiration_date'][n+3],
            'license_category': new_main_for_api['license_category'][n+3]
            },
              
              
              {
            'license_number': new_main_for_api['license_number'][n+4],
            'rollup_id': new_main_for_api['rollup_id'][n+4],
            'legal_name': new_main_for_api['legal_name'][n+4],
            'license_status': new_main_for_api['license_status'][n+4],
            'is_current': is_current_5,
            'license_description': new_main_for_api['license_description'][n+4],
            'us_state': new_main_for_api['us_state'][n+4],
            'expiration_date': new_main_for_api['expiration_date'][n+4],
            'license_category': new_main_for_api['license_category'][n+4]
            },
              
              
              {
            'license_number': new_main_for_api['license_number'][n+5],
            'rollup_id': new_main_for_api['rollup_id'][n+5],
            'legal_name': new_main_for_api['legal_name'][n+5],
            'license_status': new_main_for_api['license_status'][n+5],
            'is_current': is_current_6,
            'license_description': new_main_for_api['license_description'][n+5],
            'us_state': new_main_for_api['us_state'][n+5],
            'expiration_date': new_main_for_api['expiration_date'][n+5],
            'license_category': new_main_for_api['license_category'][n+5]
            },

              
              {
            'license_number': new_main_for_api['license_number'][n+6],
            'rollup_id': new_main_for_api['rollup_id'][n+6],
            'legal_name': new_main_for_api['legal_name'][n+6],
            'license_status': new_main_for_api['license_status'][n+6],
            'is_current': is_current_7,
            'license_description': new_main_for_api['license_description'][n+6],
            'us_state': new_main_for_api['us_state'][n+6],
            'expiration_date': new_main_for_api['expiration_date'][n+6],
            'license_category': new_main_for_api['license_category'][n+6]
            },
              
              
              {
            'license_number': new_main_for_api['license_number'][n+7],
            'rollup_id': new_main_for_api['rollup_id'][n+7],
            'legal_name': new_main_for_api['legal_name'][n+7],
            'license_status': new_main_for_api['license_status'][n+7],
            'is_current': is_current_8,
            'license_description': new_main_for_api['license_description'][n+7],
            'us_state': new_main_for_api['us_state'][n+7],
            'expiration_date': new_main_for_api['expiration_date'][n+7],
            'license_category': new_main_for_api['license_category'][n+7]
            },
              
              
              {
            'license_number': new_main_for_api['license_number'][n+8],
            'rollup_id': new_main_for_api['rollup_id'][n+8],
            'legal_name': new_main_for_api['legal_name'][n+8],
            'license_status': new_main_for_api['license_status'][n+8],
            'is_current': is_current_9,
            'license_description': new_main_for_api['license_description'][n+8],
            'us_state': new_main_for_api['us_state'][n+8],
            'expiration_date': new_main_for_api['expiration_date'][n+8],
            'license_category': new_main_for_api['license_category'][n+8]
            },

              
              {
            'license_number': new_main_for_api['license_number'][n+9],
            'rollup_id': new_main_for_api['rollup_id'][n+9],
            'legal_name': new_main_for_api['legal_name'][n+9],
            'license_status': new_main_for_api['license_status'][n+9],
            'is_current': is_current_10,
            'license_description': new_main_for_api['license_description'][n+9],
            'us_state': new_main_for_api['us_state'][n+9],
            'expiration_date': new_main_for_api['expiration_date'][n+9],
            'license_category': new_main_for_api['license_category'][n+9]
            },

              
          ],
        },
            headers={'x-api-key': 'ASYNC-SERVER-API-KEY-6Lfy84qn9ew93ZwMXSYhdwqj'}
#             {'x-api-key': '80030179832729980933'}
        )
        
        num_of_lic_sent += 10
        
        sleep(5)
        
        if len(new_main_for_api) - num_of_lic_sent < 10:
            
            for n in range(num_of_lic_sent, len(new_main_for_api)):
    
                if new_main_for_api['is_current'].iloc[n] == 1:
                    is_current_1 = True
                else:
                    is_current_1 = False
                    
                    
            requests.post(
#                 'https://bespoke-api-server-staging.herokuapp.com/api/v1/licenses/bulk_update_licenses', 
                'https://bespoke-api-server-production.herokuapp.com/api/v1/licenses/bulk_update_licenses',
                json= {
                'company_licenses': [
              
                                      {
                                    'license_number': new_main_for_api['license_number'].iloc[n],
                                    'rollup_id': new_main_for_api['rollup_id'].iloc[n],
                                    'Legal Name': new_main_for_api['legal_name'].iloc[n],
                                    'legal_name': new_main_for_api['license_status'].iloc[n],
                                    'is_current': is_current_1,
                                    'license_description': new_main_for_api['license_description'].iloc[n],
                                    'us_state': new_main_for_api['us_state'].iloc[n],
                                    'expiration_date': new_main_for_api['expiration_date'].iloc[n],
                                    'license_category': new_main_for_api['license_category'].iloc[n]
                                    },
                                  ],
                                },
                                    headers={'x-api-key': 'ASYNC-SERVER-API-KEY-6Lfy84qn9ew93ZwMXSYhdwqj'}
#                                             {'x-api-key': '80030179832729980933'}
                                )
        print(num_of_lic_sent)

In [ ]:
new_main_for_api

In [ ]:
print(stop)

### Code to Double Check the Metabase Data

In [ ]:
all_main = pd.read_json('../sweat_equity/query_result_2021-09-20T21_51_21.859Z.json')

In [ ]:
all_main

In [ ]:
all_main = all_main[~all_main['Us State'].isna()]

In [ ]:
all_main_or = all_main[all_main['Us State'] == 'OR']

In [ ]:
all_main_or = all_main_or.reset_index().drop(columns='index')

In [ ]:
all_main_or.columns

In [ ]:
to_alter = []

for lic in or_sql_data['license_number']:
    if list(or_sql_data[or_sql_data['license_number'] == lic]['license_description'])[0] != list(all_main_or[all_main_or['License Number'] == lic]['License Description'])[0]:
        print(lic, 'description')
        to_alter.append(lic)
    if list(or_sql_data[or_sql_data['license_number'] == lic]['license_category'])[0] != list(all_main_or[all_main_or['License Number'] == lic]['License Category'])[0]:
        print(lic, 'category')
        to_alter.append(lic)
    if list(or_sql_data[or_sql_data['license_number'] == lic]['legal_name'])[0] != list(all_main_or[all_main_or['License Number'] == lic]['Legal Name'])[0]:
        print(lic, 'name')
        to_alter.append(lic)
    if list(or_sql_data[or_sql_data['license_number'] == lic]['is_current'])[0] != list(all_main_or[all_main_or['License Number'] == lic]['Is Current'])[0]:
        print(lic, 'current')
        to_alter.append(lic)
        
df_for_alter = df_edited_current[df_edited_current['license_number'].isin(to_alter)]